In [16]:
import os

import dotenv
import pydantic
from langchain.agents import AgentExecutor, initialize_agent, AgentType
from langchain.agents.tools import Tool
from langchain import LLMMathChain
from langchain.chat_models import ChatOpenAI

# assuming .env in root of repo contains OPENAI_API_KEY
dotenv.load_dotenv("../../../../../.env")

llm = ChatOpenAI(temperature=0)
llm_math_chain = LLMMathChain(llm=llm, verbose=True)

Define tools which provide:
- The `n`th prime number (using a small subset for this example) 
- The LLMMathChain to act as a calculator

In [62]:
primes = {
    998:7901,
    999:7907,
    1000:7919
}

class CalculatorInput(pydantic.BaseModel):
    question: str = pydantic.Field()

class PrimeInput(pydantic.BaseModel):
    n: int = pydantic.Field()

def get_prime(n: int, primes: dict = primes) -> str:
    return str(primes.get(int(n)))

async def aget_prime(n: int, primes: dict = primes) -> str:
    return str(primes.get(int(n)))
        
tools = [
    Tool(
        name="GetPrime",
        func=get_prime, 
        description="A tool that returns the `n`th prime number",
        args_schema=PrimeInput,
        coroutine=aget_prime,
    ),
    Tool.from_function(
        func=llm_math_chain.run,
        name="Calculator",
        description="useful for when you need to answer questions about math",
        args_schema=CalculatorInput,
        coroutine=llm_math_chain.arun
    )
]

In [63]:
# Construct the agent. We will use the default agent type here.
# See documentation for a full list of options.
agent = initialize_agent(
    tools,
    llm,
    agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION,
    verbose=True
)

In [64]:
question1 = "What are the 998th, 999th and 1000th prime numbers?"
question2 = (
    "What is the product of the 998th, 999th and 1000th prime numbers?"
)

In [ ]:
for ix, step_output in enumerate(agent.iter(question1)):
    print(f"STEP {ix}:")
    print(step_output)



> Entering new  chain...
I can use the GetPrime tool to find the prime numbers.
Action: GetPrime
Action Input: 998
Observation: 7901
Thought:STEP 0:
{'intermediate_steps': [(AgentAction(tool='GetPrime', tool_input='998', log='I can use the GetPrime tool to find the prime numbers.\nAction: GetPrime\nAction Input: 998'), '7901')]}
I have found the 998th prime number. Now I need to find the 999th and 1000th prime numbers.
Action: GetPrime
Action Input: 999
Observation: 7907
Thought:STEP 1:
{'intermediate_steps': [(AgentAction(tool='GetPrime', tool_input='998', log='I can use the GetPrime tool to find the prime numbers.\nAction: GetPrime\nAction Input: 998'), '7901'), (AgentAction(tool='GetPrime', tool_input='999', log='I have found the 998th prime number. Now I need to find the 999th and 1000th prime numbers.\nAction: GetPrime\nAction Input: 999'), '7907')]}


In [61]:
for ix, step_output in enumerate(agent.iter(question2)):
    print(f"STEP {ix}:")
    print(step_output)
    



> Entering new  chain...
To find the product of the 998th, 999th, and 1000th prime numbers, I can use the GetPrime tool to find each prime number and then multiply them together.
Action: GetPrime
Action Input: 998
Observation: None
Thought:STEP 0:
{'intermediate_steps': [(AgentAction(tool='GetPrime', tool_input='998', log='To find the product of the 998th, 999th, and 1000th prime numbers, I can use the GetPrime tool to find each prime number and then multiply them together.\nAction: GetPrime\nAction Input: 998'), 'None')]}
I need to find the 998th prime number.
Action: GetPrime
Action Input: 998
Observation: None
Thought:STEP 1:
{'intermediate_steps': [(AgentAction(tool='GetPrime', tool_input='998', log='To find the product of the 998th, 999th, and 1000th prime numbers, I can use the GetPrime tool to find each prime number and then multiply them together.\nAction: GetPrime\nAction Input: 998'), 'None'), (AgentAction(tool='GetPrime', tool_input='998', log='I need to find the 998th pri

In [21]:
from functools import reduce

In [22]:
reduce(lambda x, y: x*y, primes.values())

494725326233

In [36]:
7901*7907*7919

494725326233

In [38]:
494725326233 / 1,492,313,491

(494725326233.0, 492, 313, 491)

In [40]:
494725326233 / 1492313491

331.5156830093952